In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pathlib import Path

In [ ]:
formulaCompressionTestHome = "C:\\Users\\jonat\\Documents\\Dataspread\\FormulaCompressionTest"

In [91]:
def getMiscDict(path):
    dirList = path.split('\\')
    syncType = dirList[len(dirList) - 7]
    memoryStorage = dirList[len(dirList) - 6]
    baseline = dirList[len(dirList) -5]
    if baseline.lower() == 'comp':
        baseline = 'TACO'
    elif baseline.lower() == 'async':
        baseline = 'Anti-freeze'
    elif baseline.lower() == 'pgimpl':
        baseline = 'NoComp'
    testSpreadsheet = dirList[len(dirList) - 4]
    run = int(dirList[len(dirList) - 3].split('-')[1])
    rows = int(dirList[len(dirList) - 2].split('-')[1])
    return {'Sync/Async':syncType, 'Memory Storage':memoryStorage, 'Baseline':baseline,
            'Test Spreadsheet':testSpreadsheet, 'Run':run, 'Rows':rows}

def getMetrics(path):
    """
    Takes in a path string to a .stat file and returns a Pandas DataFrame of the file's contents
    """
    reportsFile = open(path, 'r')
    dataDict = {}
    dataDict = getMiscDict(path)
    for line in reportsFile:
        if ': ' in line:
            metric, value = line.split(": ")
            if metric not in dataDict:
                dataDict[metric] = []
            try:
                dataDict[metric].append(int(value))
            except ValueError:
                dataDict[metric].append(float(value))
            except:
                dataDict[metric].append(value)
    reportsFile.close
    
    return pd.DataFrame(dataDict)


def getAllMetrics(basePath, miscDict={'sync type':'', 'mem type':'', 'impl':'', 'test':'', 'run':0}):
    """
    Takes in a file path and searches all of its subdirectories for .stat files and returns all of
    their contents in a Pandas DataFrame
    """
    data = pd.DataFrame()
    for file in os.listdir(basePath):
        filePath = os.path.join(basePath, file)
        if os.path.isdir(filePath):
            data = data.append(getAllMetrics(filePath))
        elif file == "core.stat":
            data = data.append(getMetrics(filePath))
    return data


           
def plotTotalTime(data, 
                  researchQuestion,
                  sheets=['Rate', 'RunningTotalFast', 'RunningTotalSlow'], 
                  memStorage = ['DB', 'IN_MEM', 'MEM'], 
                  syncMethod=['async', 'sync'], 
                  showTitle=True, 
                  outerLegend=True,
                  save=False):
    
    plt.rcParams.update({'font.size': 25, 'figure.figsize': (12,8), 'lines.linewidth':3, 'axes.grid':True})
    for sheet in sheets:
        for sync in syncMethod:
            for mem in memStorage:
                dataSlice = pd.DataFrame()
                try:
                    dataSlice = data.loc[sheet, mem, sync]
                except:
                    print('No data for (' + sheet + ', ' + mem + ', ' + sync + ')')
                
                if len(dataSlice > 0):
                    relevantData = pd.pivot_table(dataSlice, 
                                              index='Rows', 
                                              columns=dataSlice.index)[['Total time to update cells (ms)', 
                                                                        'Total time of adding the batch (ms)']]
                    splitData = [relevantData[relevantData.index < 100000], relevantData[relevantData.index >= 100000]]
                    if len(splitData[1]) == 0:
                        splitData = [splitData[0]]
                    for plotData in splitData:
                        plt.figure()
                        totalTestTimeData = plotData['Total time to update cells (ms)']
                        totalTestTimePlot = sns.lineplot(markers=True, 
                                                         markersize=20, 
                                                         ci=None, 
                                                         dashes=False, 
                                                         alpha=.5, 
                                                         color="black", 
                                                         data=totalTestTimeData)
                        if (outerLegend):
                            handles, labels = totalTestTimePlot.get_legend_handles_labels()
                            leg = totalTestTimePlot.legend(
                                bbox_to_anchor=(0, 1, 1, 1),
                                ncol=3,
                                mode="expand",
                                loc="lower left",
                                handles=handles[0:], 
                                labels=labels[0:],
                                markerscale=3
                                )
                        if sync=='sync':
                            if showTitle:
                                totalTestTimePlot.set_title('Time to Update Spreadsheet (' + sheet + ', ' + mem + ', ' + sync + ')', pad=70)
                            totalTestTimePlot.set_ylabel('Time to Update Spreadsheet (ms)')
                        elif sync=='async':
                            if showTitle:
                                totalTestTimePlot.set_title('Time to Return Control to User (' + sheet + ', ' + mem + ', ' + sync + ')', pad=70)
                            totalTestTimePlot.set_ylabel('Time to Return Control to User (ms)')
                        if save: 
                            savePlot(totalTestTimePlot, researchQuestion, sheet, mem, sync, (min(plotData.index),  max(plotData.index)), showTitle, outerLegend)
                            plt.close()

                        
def plotBatchLoadingTime(data, 
                         researchQuestion,
                         sheets=['Rate', 'RunningTotalFast', 'RunningTotalSlow'], 
                         memStorage = ['DB', 'IN_MEM', 'MEM'], 
                         syncMethod=['async', 'sync'], 
                         showTitle=True, 
                         outerLegend=True,
                         save=False):
    
    plt.rcParams.update({'font.size': 25, 'figure.figsize': (12,8), 'lines.linewidth':3, 'axes.grid':True})
    for sheet in sheets:
        for sync in syncMethod:
            for mem in memStorage:
                dataSlice = pd.DataFrame()
                try:
                    dataSlice = data.loc[sheet, mem, sync]
                except:
                    print('No data for (' + sheet + ', ' + mem + ', ' + sync + ')')
                
                if len(dataSlice > 0):
                    relevantData = pd.pivot_table(dataSlice, 
                                              index='Rows', 
                                              columns=dataSlice.index)[['Total time to update cells (ms)', 
                                                                        'Total time of adding the batch (ms)']]
                    splitData = [relevantData[relevantData.index < 100000], relevantData[relevantData.index >= 100000]]
                    if len(splitData[1]) == 0:
                        splitData = [splitData[0]]
                    for plotData in splitData:
                        plt.figure()
                        batchData = plotData['Total time of adding the batch (ms)']
                        yMax = batchData['NoComp'].max()
                        cappedData = batchData.copy()
                        for column in batchData.columns:
                            cappedData[column] = batchData[column].map(lambda x: min(x, yMax * 1.05))
                        batchPlot = sns.lineplot(markers=True, markersize=20, ci=None, dashes=False, alpha=.5, color="black", data=cappedData)
                        xAdjust = - max(batchData.index.values) * .03
                        yAdjust = yMax * .05
                        xMax = max(batchData.index.values)
                        for column in batchData.columns:
                            for i, row in batchData[batchData[column] != cappedData[column]].iterrows():
                                xMax = max(batchData.index.values) * 1.14
                                batchPlot.text(x=int(row.name) + xAdjust, 
                                               y=cappedData.loc[row.name, column] + yAdjust, 
                                               s='{:.1e}'.format(row[column]))
                        if outerLegend:
                            handles, labels = batchPlot.get_legend_handles_labels()
                            leg = batchPlot.legend(
                                bbox_to_anchor=(0, 1.03, 1, 1),
                                ncol=3,
                                mode="expand",
                                loc="lower left",
                                handles=handles[0:], 
                                labels=labels[0:],
                                markerscale=3
                            )
                            batchPlot.set_ybound(0 - yMax * .05, yMax * 1.2)
                            batchPlot.set_xbound(upper=xMax)
                        if showTitle:
                            batchPlot.set_title('Batch Loading Time (' + sheet + ')', pad=80) #rename to loading Time
                        batchPlot.set_ylabel('Total Time (ms)')
                        if save: 
                            savePlot(batchPlot, researchQuestion, sheet, mem, sync, (min(plotData.index),  max(plotData.index)), showTitle, outerLegend)
                            plt.close()
                        

def savePlot(plot, researchQuestion, sheet, mem, sync, plotRange, showTitle, outerLegend):
    fileString = '../plots/Dixin-Exp'+'/'+researchQuestion+'/'+sheet+'/'+mem+'/'
    Path(fileString).mkdir(parents=True, exist_ok=True)
    if showTitle:
        fileString += 'WithTitle_'
    else:
        fileString += 'WithoutTitle_'
    if outerLegend:
        fileString += 'WithOuterLegend_'
    else:
        fileString += 'WithInnerLegend_'
    
    fileString += '(' + str(plotRange[0]) + ',' + str(plotRange[1]) + ')' + '.pdf'
    plt.savefig(fileString, bbox_inches="tight")
    print('Plot saved to ' + fileString)
    
    
def plotBars(testData, numRows, savePlot=False):
    relevantData = testData[testData['Rows'] == numRows]
    try:
        relevantData = relevantData.loc[:, 'MEM', 'async']
    except:
        relevantData = relevantData.loc[:, 'IN_MEM', 'async']
    relevantData = relevantData[['Total time to update cells (ms)']]
    relevantData = relevantData.reset_index(level=['Test Spreadsheet', 'Baseline']).pivot(index='Test Spreadsheet', columns='Baseline', values='Total time to update cells (ms)')
    colors = []
    if 'Anti-freeze' in relevantData.columns:
        colors.append('tab:blue')
    if 'NoComp' in relevantData.columns:
        colors.append('tab:orange')
    if 'TACO' in relevantData.columns:
        colors.append('tab:green')
    ax = relevantData.plot.bar(color=colors)
    ax.set_title('Returning Control to Users (Mem)', pad=70)
    ax.set_ylabel('Time (ms)')
    handles, labels = ax.get_legend_handles_labels()
    leg = ax.legend(
                    bbox_to_anchor=(0, 1, 1, 1),
                    ncol=3,
                    mode="expand",
                    loc="lower left",
                    handles=handles[0:], 
                    labels=labels[0:],
                    markerscale=3
                    )
    if savePlot:
        filePath = '../plots/Dixin-Exp/rq1/MemBarPlots/'
        Path(filePath).mkdir(parents=True, exist_ok=True)
        plt.savefig(filePath + 'ReturningControltoUsersBarChart_' + str(numRows) + '.pdf', bbox_inches="tight")
        plt.close()

In [92]:
#RQ1

formulaCompressionTestHome = "C:\\Users\\jonat\\Documents\\Dataspread\\FormulaCompressionTest"
#testPath = formulaCompressionTestHome + "\\Chris\\FormulaCompressionTesting\\testing\\reports"
testPath = formulaCompressionTestHome + "\\experiments_data\\rq1\\report"
pd.set_option('max_rows', 115)
testData = getAllMetrics(testPath)
testData = testData.groupby(['Test Spreadsheet', 'Memory Storage', 'Sync/Async', 'Baseline', 'Rows']).median()
testData = testData.reset_index(level='Rows')
#display(testData)

#plotTotalTime(testData, researchQuestion='rq1', memStorage=['DB'], showTitle=True, outerLegend=True, save=False)

plotTotalTime(testData, researchQuestion='rq1', memStorage=['DB'], showTitle=True, outerLegend=True, save=True)
plotTotalTime(testData, researchQuestion='rq1', memStorage=['DB'], showTitle=True, outerLegend=False, save=True)
plotTotalTime(testData, researchQuestion='rq1', memStorage=['DB'], showTitle=False, outerLegend=True, save=True)
plotTotalTime(testData, researchQuestion='rq1', memStorage=['DB'], showTitle=False, outerLegend=False, save=True)





#testPath = formulaCompressionTestHome + "\\Chris\\FormulaCompressionTesting\\testing\\reports"
testData = getAllMetrics(testPath)
testData = testData.groupby(['Test Spreadsheet', 'Memory Storage', 'Sync/Async', 'Baseline', 'Rows']).median()
testData = testData.reset_index(level='Rows')
#display(testData)
plotBars(testData, 10000, savePlot=True)
plotBars(testData, 500000, savePlot=True)

Plot saved to ../plots/Dixin-Exp/rq1/Rate/DB/WithTitle_WithOuterLegend_(5000,25000).pdf
Plot saved to ../plots/Dixin-Exp/rq1/Rate/DB/WithTitle_WithOuterLegend_(100000,500000).pdf
No data for (Rate, DB, sync)
Plot saved to ../plots/Dixin-Exp/rq1/RunningTotalFast/DB/WithTitle_WithOuterLegend_(5000,25000).pdf
Plot saved to ../plots/Dixin-Exp/rq1/RunningTotalFast/DB/WithTitle_WithOuterLegend_(100000,500000).pdf
No data for (RunningTotalFast, DB, sync)
Plot saved to ../plots/Dixin-Exp/rq1/RunningTotalSlow/DB/WithTitle_WithOuterLegend_(5000,25000).pdf
Plot saved to ../plots/Dixin-Exp/rq1/RunningTotalSlow/DB/WithTitle_WithOuterLegend_(100000,500000).pdf
No data for (RunningTotalSlow, DB, sync)
Plot saved to ../plots/Dixin-Exp/rq1/Rate/DB/WithTitle_WithInnerLegend_(5000,25000).pdf
Plot saved to ../plots/Dixin-Exp/rq1/Rate/DB/WithTitle_WithInnerLegend_(100000,500000).pdf
No data for (Rate, DB, sync)
Plot saved to ../plots/Dixin-Exp/rq1/RunningTotalFast/DB/WithTitle_WithInnerLegend_(5000,25000).

In [93]:
#RQ2

formulaCompressionTestHome = "C:\\Users\\jonat\\Documents\\Dataspread\\FormulaCompressionTest"
testPath = formulaCompressionTestHome + "\\experiments_data\\rq2\\report"
pd.set_option('max_rows', 115)
testData = getAllMetrics(testPath)
testData = testData.groupby(['Test Spreadsheet', 'Memory Storage', 'Sync/Async', 'Baseline', 'Rows']).median()
testData = testData.reset_index(level='Rows')
#display(testData)

#plotBatchLoadingTime(testData, researchQuestion='rq2', memStorage=['DB'], showTitle=True, outerLegend=True, save=False)


plotBatchLoadingTime(testData, researchQuestion='rq2', showTitle=True, outerLegend=True, save=True)
plotBatchLoadingTime(testData, researchQuestion='rq2', showTitle=True, outerLegend=False, save=True)
plotBatchLoadingTime(testData, researchQuestion='rq2', showTitle=False, outerLegend=True, save=True)
plotBatchLoadingTime(testData, researchQuestion='rq2', showTitle=False, outerLegend=False, save=True)

posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values


Plot saved to ../plots/Dixin-Exp/rq2/Rate/DB/WithTitle_WithOuterLegend_(5000,25000).pdf
Plot saved to ../plots/Dixin-Exp/rq2/Rate/DB/WithTitle_WithOuterLegend_(100000,500000).pdf
No data for (Rate, IN_MEM, async)
No data for (Rate, MEM, async)
No data for (Rate, DB, sync)
No data for (Rate, IN_MEM, sync)
No data for (Rate, MEM, sync)


posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values


Plot saved to ../plots/Dixin-Exp/rq2/RunningTotalFast/DB/WithTitle_WithOuterLegend_(5000,25000).pdf
Plot saved to ../plots/Dixin-Exp/rq2/RunningTotalFast/DB/WithTitle_WithOuterLegend_(100000,500000).pdf
No data for (RunningTotalFast, IN_MEM, async)
No data for (RunningTotalFast, MEM, async)
No data for (RunningTotalFast, DB, sync)
No data for (RunningTotalFast, IN_MEM, sync)
No data for (RunningTotalFast, MEM, sync)


posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values


...

Plot saved to ../plots/Dixin-Exp/rq2/Rate/DB/WithoutTitle_WithInnerLegend_(5000,25000).pdf
Plot saved to ../plots/Dixin-Exp/rq2/Rate/DB/WithoutTitle_WithInnerLegend_(100000,500000).pdf
No data for (Rate, IN_MEM, async)
No data for (Rate, MEM, async)
No data for (Rate, DB, sync)
No data for (Rate, IN_MEM, sync)
No data for (Rate, MEM, sync)


posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values


Plot saved to ../plots/Dixin-Exp/rq2/RunningTotalFast/DB/WithoutTitle_WithInnerLegend_(5000,25000).pdf
Plot saved to ../plots/Dixin-Exp/rq2/RunningTotalFast/DB/WithoutTitle_WithInnerLegend_(100000,500000).pdf
No data for (RunningTotalFast, IN_MEM, async)
No data for (RunningTotalFast, MEM, async)
No data for (RunningTotalFast, DB, sync)
No data for (RunningTotalFast, IN_MEM, sync)
No data for (RunningTotalFast, MEM, sync)


posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values


Plot saved to ../plots/Dixin-Exp/rq2/RunningTotalSlow/DB/WithoutTitle_WithInnerLegend_(5000,25000).pdf
Plot saved to ../plots/Dixin-Exp/rq2/RunningTotalSlow/DB/WithoutTitle_WithInnerLegend_(100000,500000).pdf
No data for (RunningTotalSlow, IN_MEM, async)
No data for (RunningTotalSlow, MEM, async)
No data for (RunningTotalSlow, DB, sync)
No data for (RunningTotalSlow, IN_MEM, sync)
No data for (RunningTotalSlow, MEM, sync)


In [95]:
#RQ3
testPath = formulaCompressionTestHome + "\\experiments_data\\rq3\\report"
pd.set_option('max_rows', 115)
testData = getAllMetrics(testPath)
testData = testData.groupby(['Test Spreadsheet', 'Memory Storage', 'Sync/Async', 'Baseline', 'Rows']).median()
testData = testData.reset_index(level='Rows')
plotTotalTime(testData, researchQuestion='rq3', sheets=['RefreshCache'], showTitle=True, outerLegend=True, save=True)
plotTotalTime(testData, researchQuestion='rq3', sheets=['RefreshCache'], showTitle=True, outerLegend=False, save=True)
plotTotalTime(testData, researchQuestion='rq3', sheets=['RefreshCache'], showTitle=False, outerLegend=True, save=True)
plotTotalTime(testData, researchQuestion='rq3', sheets=['RefreshCache'], showTitle=False, outerLegend=False, save=True)

Plot saved to ../plots/Dixin-Exp/rq3/RefreshCache/DB/WithTitle_WithOuterLegend_(2000,10000).pdf
No data for (RefreshCache, IN_MEM, async)
Plot saved to ../plots/Dixin-Exp/rq3/RefreshCache/MEM/WithTitle_WithOuterLegend_(2000,10000).pdf
No data for (RefreshCache, DB, sync)
No data for (RefreshCache, IN_MEM, sync)
No data for (RefreshCache, MEM, sync)
Plot saved to ../plots/Dixin-Exp/rq3/RefreshCache/DB/WithTitle_WithInnerLegend_(2000,10000).pdf
No data for (RefreshCache, IN_MEM, async)
Plot saved to ../plots/Dixin-Exp/rq3/RefreshCache/MEM/WithTitle_WithInnerLegend_(2000,10000).pdf
No data for (RefreshCache, DB, sync)
No data for (RefreshCache, IN_MEM, sync)
No data for (RefreshCache, MEM, sync)
Plot saved to ../plots/Dixin-Exp/rq3/RefreshCache/DB/WithoutTitle_WithOuterLegend_(2000,10000).pdf
No data for (RefreshCache, IN_MEM, async)
Plot saved to ../plots/Dixin-Exp/rq3/RefreshCache/MEM/WithoutTitle_WithOuterLegend_(2000,10000).pdf
No data for (RefreshCache, DB, sync)
No data for (Refresh